In [ ]:
import pandas as pd
import urllib.request
import numpy as np
from IPython.display import display
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
# to make this notebook's output stable across runs
np.random.seed(42)

In [ ]:
print('Loading case data ...')
cases = pd.read_csv("pancancer_case_features_100.csv")
print("done.")

**Question:** If we opened the 100 case features file, why does the table below show 1063 columns. Also, I think we should encode the labels, perform PCA and select the number of features in the previous notebook.

In [ ]:
labels_string = cases.cancer_type
le = preprocessing.LabelEncoder()
labels = le.fit_transform(labels_string)

# Get rid of the cancer type and patient_id columns 
data = cases[cases.columns[3:]]

display(labels)
display(data.head())

As discussed before, we have a LOT of features. However, we know that most genes are not active for our dataset. Let us perform a dimensionality reduction exercise with our data. **Note:** I am performing this with the entire dataset. However, we should do this only with the training dataset.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(data)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1
print("PCA suggested dimension: %d" %d)

In [ ]:
#skf = StratifiedKFold(n_splits=10)
#for train_index, test_index in skf.split(data, labels):
#    train_data, test_data     = data.values[train_index], data.values[test_index]
#    train_labels, test_labels = labels[train_index], labels[test_index]
#    print(len(train_data), len(test_data))
    



In [ ]:
data.shape

Here we split our data into three categories. First we split the data into *training data* and *test data*. For this, we use the common splitting rule of 70:30. We take care to *stratify* the separation based on the labels because we have seen that some cancer types are extremely sparsely represented. Next, we create a *development* dataset used for tuning the hyperparameters of the various models we plan to try. Here, we use separate **20%** of the testing data for development purposes.

In [ ]:
train_data_all, test_data, train_labels_all, test_labels = train_test_split(data, labels,
                                                           stratify=labels, 
                                                           test_size=0.30)

train_data, dev_data, train_labels, dev_labels = train_test_split(train_data_all, train_labels_all,
                                                                 stratify=train_labels_all, 
                                                                 test_size=0.20)

print("training data:", train_data.shape)
print("dev data     :", dev_data.shape)
print("test data    :",  test_data.shape)

## Multinomial Naive Bayes ##

First, we try the NaiveBayes classifier. NaiveBayes is a generative model that works by using the training data to create a probability distribution of the different features for the different labels. For this, it simply uses the frequency of the feature conditional on the label from the training dataset. It then uses the independence assumption of these features conditional on the label in the Bayes rule to compute the probability of a given sample in the test dataset. Because of the assumption of independence (which may be questionable for cancer data, even conditional on the label), the probability calculation is simply the normalized product of learnt feature probabilities for each label. The classifier then simply predicts the label with the highest computed probability.

Since some features may be completely absent in the training dataset for some labels, we use Laplace smoothing to ensure that the probability product is not zero. This is represented by the hyperparameter **alpha** of the classifier. We use *GridSearchCV* using a cross-validation size of **5** to find the most optimal value for this parameter.

**Question:** If we are using GridSearchCV, should we still create a dev dataset?

In [ ]:
#
# Multinomial Naive Bayes
#
mnb = MultinomialNB()
params = {'alpha': [0.001, 0.01, 0.1, 0.5, 1, 10]}
mnb = GridSearchCV(mnb, params, cv=5,
                   scoring='accuracy', return_train_score=True)
# Fit  training data
mnb.fit(train_data, train_labels)  
mnb_best_alpha = mnb.best_params_
mnb_cv_accuracy = np.round(mnb.best_score_, 4)

# Show the best alpha parameter to use and the expected accuracy
print('\nMultinomial Naive Bayes Classifier')
print(' Best param (alpha):\t',   mnb_best_alpha)
print(' Accuracy (CV): \t',   mnb_cv_accuracy)

The GridSearch reports back an optimial alpha of 1 and a CV accuracy on the training set of **44%**. We now test these parameters on the test dataset.

In [ ]:
mnb = MultinomialNB(1)
mnb.fit(train_data, train_labels)
mnb_pred = mnb.predict(test_data)

#plt.plot(fpr, tpr, "b:", label="SGD")
#plot_roc_curve(fpr_forest, tpr_forest, "Random Forest")
#plt.legend(loc="lower right")
#plt.show()
print("Test set accuracy:\t", metrics.accuracy_score(test_labels, mnb_pred))
print("\n")
print(metrics.classification_report(test_labels, mnb_pred))

In [ ]:
#
# Logistic Regression
#
lr = LogisticRegression(penalty='l2', multi_class = 'ovr', solver='liblinear', max_iter=150)
params = {'C': [0.001, 0.01, 0.1, 0.5, 1, 10]}
logit = GridSearchCV(lr, params, cv=5,
                     scoring='accuracy', return_train_score=True)

# Fit  training data
logit.fit(train_data, train_labels)  
# Show the best C parameter to use and the expected accuracy
print('\nLogistic Regression Classifier, L2 regularization')
print(' Best param:', logit.best_params_)
print(' Accuracy:  ', np.round(logit.best_score_, 4))

In [ ]:
def runLogitL1(train_data, train_labels, dev_data, dev_labels, c_param):
    l1 = LogisticRegression(penalty='l1', tol=.01, 
                            solver="liblinear", multi_class="ovr",
                            max_iter=500, C=c_param)
    # Fit model
    l1.fit(train_data, train_labels) 
    # Predict
    predict = l1.predict(dev_data)
    # Get precision, recall, f1 scores
    scores = precision_recall_fscore_support(dev_labels, predict, average='weighted', labels=np.unique(predict))  
    
    # Get the features with non-zero coefficients.  We will use
    # this list to reduce the features for the
    # following logistic regression with L2 regularization
    non_zero_sums = np.where(np.sum(l1.coef_, axis=0) != 0)
    names = np.array(list(train_data.columns))
    non_zero_names = names[non_zero_sums] 
    
    return {'scores': scores, 'non_zero_genes': non_zero_names}

from textwrap import wrap
params = {'C':  [1000, 100, 10, 1, .5, .3, .1, .05]}


# Now perform logistic regression on this training set with reduced features
# as well as the orginal non-reduced training set.  Run over different
# C values to plot differences in accuracy
precision_l1        = []
recall_l1           = []
precision_l2      = []
recall_l2           = []
feature_size        = []


for c_param in reversed(params['C']):
    # Keep this random seed here to make comparison easier.
    np.random.seed(0)

    #
    # Perform Logistic Regression on different C values
    # using L1 regularization
    #
    l1_info = runLogitL1(train_data, train_labels, dev_data, dev_labels, c_param)    
    non_zero_genes = l1_info['non_zero_genes']
    feature_size.append(str(len(non_zero_genes)) + ' (C=' + str(c_param) + ")")
    precision_l1.append(l1_info['scores'][0])
    recall_l1.append(l1_info['scores'][1])


    #
    # Reduce feature size, only keeping features with non-zero weights 
    # found using l1 regularization
    #
    min_train_data = train_data[non_zero_genes]
    min_dev_data   = dev_data[non_zero_genes]


    # Run logistic regression with L2 regularization on reduced
    # feature set
    lr = LogisticRegression(penalty='l2', tol=.01, max_iter=150, 
                            C=logit.best_params_['C'], solver="liblinear", multi_class="ovr")
    lr.fit(min_train_data, train_labels) 

    predict = lr.predict(min_dev_data)
    
    # Get precision, recall, f1 scores
    scores = precision_recall_fscore_support(dev_labels, predict, average='weighted', labels=np.unique(predict))  
    precision_l2.append(scores[0])
    recall_l2.append(scores[1])






In [ ]:

labels = [ '\n'.join(wrap(l, 8)) for l in feature_size ]        
plt.rcParams["figure.figsize"] = (20,20)
plt.plot(labels, precision_l1, color='mediumblue',
         linewidth=3, label='L1 precision at different C values', marker='o' )
plt.plot(labels, recall_l1, color='mediumblue', linestyle='dashed',
         linewidth=3, label='L1 at different C values', marker='o' )

plt.plot(labels, precision_l2, color='darkorange', 
         linewidth=3, label='L2 precision with reduced number of features', marker='o' )
plt.plot(labels, recall_l2, color='darkorange', linestyle='dashed',
         linewidth=3, label='L2 recall with reduced number of features', marker='o' )

plt.yticks(np.arange(.34, .8, .01))
plt.ylabel('Precision, Recall', fontsize=20)
plt.xlabel('Feature size with L1 regularization at different C parameters', fontsize=20, labelpad=20)
plt.legend()
plt.grid()
plt.show()
        